# Testing openpyxl insert capabilities

This is a repo to test the capabilies of OpenPyXL to insert data into an existing excel fila (template.xlsx)

STEPS:

1. Initialization
2. Load Pickle file
3. Calculate column "Duracion" and male sure it is in the correct place
4. Load template and insert data. 

## 1. Initilization

In [ ]:
import pandas as pd
from openpyxl import load_workbook
from openpyxl.utils.dataframe import dataframe_to_rows
import shutil
import datetime
import os


## 2. Load Pickle FIle

In [24]:
# Hardcoded paths (relative to project root)
template_path = 'templates/template.xlsx'  # Your template file
pickle_path = 'pickles/data.pkl'          # Your pickle file

# Generate output filename with date prefix (e.g., 20240209_template.xlsx)
today = datetime.date.today().strftime('%Y%m%d')
output_filename = f"{today}_template.xlsx"
output_path = os.path.join('output', output_filename)

# Copy template to output folder
shutil.copyfile(template_path, output_path)
print(f"Template copied to {output_path}")

# Load Pandas DataFrame from pickle
df = pd.read_pickle(pickle_path)
df['uuid'] = df['uuid'].astype(str)


print(f"Loaded DataFrame with {len(df)} rows")

Template copied to output/20260209_template.xlsx
Loaded DataFrame with 2721 rows


## 3. Calculate duracion

In [25]:
df['Duracion'] = df.apply(
    lambda row: (pd.to_datetime(row['FinEvento']) - pd.to_datetime(row['InicioEvento'])).total_seconds() / 60,
    axis=1
)
df['Duracion'] = df['Duracion'].round(0).astype(int)
cols = df.columns.tolist()
duracion_index = cols.index('Duracion')  # Find where we temporarily added it
cols.insert(15, cols.pop(duracion_index))  # Move it to position 14
df = df[cols]

In [27]:
df.sample(3)

,Item,Cuenta,Evento,Actividad,Alimentador,Primario,Desconexion,SIG,Tipo,Materiales,...,FinEvento,Duracion,Responsable,Colaboradores,HorasExtra,Vehiculo,Sitio,id_ot,Archivo,uuid
1913,10,lunch,Lunch en Las Peñas,ALIMEN,·,No,No,No,LUNCH,·,...,2025-03-21 14:10:00,60,GUZMAN BARROS MARCO FERNANDO,1,No,4-52,"Gualaquiza, Portón, Roldós, Bellavista.",149633,Orden de trabajo Gualaquiza 21-03-2025 (FG).pdf,b7bceaed-b771-41ce-998e-245f895a0463
2561,12,MEDIDORES,"Chuin Felix, estructura# 237813, Nuevo Servici...",PROG,Bomboiza,No,No,No,EXPANSION,·,...,2025-03-07 15:50:00,40,POGO JACOME DIEGO LEONARDO,1,No,4-74,"Chumpias, Rio Napo, Pumpuis, Chuin Felix.",148722,Orden de trabajo Gualaquiza 07-03-2025 (DP - E...,00f6dd9e-2896-4cce-a47c-2261f270249f
184,5,transporte,Nos trasladamos desde Santa Elena a la agencia...,TRANSP,·,No,No,No,TRANSPORTE,·,...,2025-03-15 13:44:00,7,SILVA ARMIJOS ROMEL EDUARDO,1,No,2-110,Zamora,149271,Orden de trabajo Zamora 15-03-2025 (RS).pdf,6e6b6a99-5ebb-419c-900d-54314991ea23


In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2721 entries, 0 to 2720
Data columns (total 24 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Item           2721 non-null   int64 
 1   Cuenta         2721 non-null   object
 2   Evento         2721 non-null   object
 3   Actividad      2721 non-null   object
 4   Alimentador    2721 non-null   object
 5   Primario       2721 non-null   object
 6   Desconexion    2721 non-null   object
 7   SIG            2721 non-null   object
 8   Tipo           2721 non-null   object
 9   Materiales     2721 non-null   object
 10  Cuadrilla      2721 non-null   object
 11  Dia            2721 non-null   object
 12  Fecha          2721 non-null   object
 13  InicioEvento   2721 non-null   object
 14  FinEvento      2721 non-null   object
 15  Duracion       2721 non-null   int64 
 16  Responsable    2721 non-null   object
 17  Colaboradores  2721 non-null   int64 
 18  HorasExtra     2721 non-null

## 4. Load the template and insert the data

In [30]:
    
# Load the copied workbook
wb = load_workbook(output_path)

# Access the second sheet (0-based index; change if needed)
sheet = wb.worksheets[1]  # Or wb['Sheet2'] if named

# Optional: Clear existing data from row 2 down (preserves headers and formats)
for row in sheet.iter_rows(min_row=2, max_row=sheet.max_row, min_col=1, max_col=sheet.max_column):
    for cell in row:
        cell.value = None

# Insert DataFrame starting from row 2 (skip headers in DF)
for r_idx, row in enumerate(dataframe_to_rows(df, index=False, header=False), 2):
    for c_idx, value in enumerate(row, 1):
        sheet.cell(row=r_idx, column=c_idx, value=value)

# Save the modified workbook
wb.save(output_path)
print(f"Data inserted into {output_path}")


Data inserted into output/20260209_template.xlsx
